In [ ]:
#https://journal.transformativeworks.org/index.php/twc/article/view/460/384
#black panther his evolution
#Data set is from Kaggle

#https://www.shanelynn.ie/summarising-aggregation-and-grouping-data-in-python-pandas/
#https://jakevdp.github.io/PythonDataScienceHandbook/03.08-aggregation-and-grouping.html
#https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.aggregate.https://www.kaggle.com/fivethirtyeight/fivethirtyeight-comic-characters-dataset

#

In [3]:
import os
import re

In [4]:
import pandas as pd
import numpy as np

In [5]:
from sqlalchemy import create_engine

In [6]:
from flask_sqlalchemy import SQLAlchemy

In [7]:
#Import csv data 

dc_path = 'dataset/dc-wikia-data.csv'
marvel_path = 'dataset/marvel-wikia-data.csv'

dc_data = pd.read_csv(dc_path)
marvel_data= pd.read_csv(marvel_path)

In [25]:
def get_name(name):
    x = re.search(r'\((.*?)\)',name)
    if x is not None:
        return(x.group().strip("()"))   
    else:
        return "nope"

def get_alias(name):
    x = re.search(r'\w+(?=\s*\([^/])',name)
    if x is not None:
        return(x.group())
    else:
        return "nope"

In [9]:
print(get_name("Batman (Bruce Wayne)"))

Bruce Wayne


In [10]:
print(get_alias("Batman (Bruce Wayne)"))

Batman


In [11]:
marvel_data.head()

,page_id,name,urlslug,ID,ALIGN,EYE,HAIR,SEX,GSM,ALIVE,APPEARANCES,FIRST APPEARANCE,Year
0,1678,Spider-Man (Peter Parker),\/Spider-Man_(Peter_Parker),Secret Identity,Good Characters,Hazel Eyes,Brown Hair,Male Characters,NaN,Living Characters,4043.0,Aug-62,1962.0
1,7139,Captain America (Steven Rogers),\/Captain_America_(Steven_Rogers),Public Identity,Good Characters,Blue Eyes,White Hair,Male Characters,NaN,Living Characters,3360.0,Mar-41,1941.0
2,64786,"Wolverine (James \""Logan\"" Howlett)",\/Wolverine_(James_%22Logan%22_Howlett),Public Identity,Neutral Characters,Blue Eyes,Black Hair,Male Characters,NaN,Living Characters,3061.0,Oct-74,1974.0
3,1868,"Iron Man (Anthony \""Tony\"" Stark)",\/Iron_Man_(Anthony_%22Tony%22_Stark),Public Identity,Good Characters,Blue Eyes,Black Hair,Male Characters,NaN,Living Characters,2961.0,Mar-63,1963.0
4,2460,Thor (Thor Odinson),\/Thor_(Thor_Odinson),No Dual Identity,Good Characters,Blue Eyes,Blond Hair,Male Characters,NaN,Living Characters,2258.0,Nov-50,1950.0


In [12]:
marvel_data.rename(columns={
    'Year':'YEAR'
},inplace=True)

In [13]:
dc_data.head()

,page_id,name,urlslug,ID,ALIGN,EYE,HAIR,SEX,GSM,ALIVE,APPEARANCES,FIRST APPEARANCE,YEAR
0,1422,Batman (Bruce Wayne),\/wiki\/Batman_(Bruce_Wayne),Secret Identity,Good Characters,Blue Eyes,Black Hair,Male Characters,NaN,Living Characters,3093.0,"1939, May",1939.0
1,23387,Superman (Clark Kent),\/wiki\/Superman_(Clark_Kent),Secret Identity,Good Characters,Blue Eyes,Black Hair,Male Characters,NaN,Living Characters,2496.0,"1986, October",1986.0
2,1458,Green Lantern (Hal Jordan),\/wiki\/Green_Lantern_(Hal_Jordan),Secret Identity,Good Characters,Brown Eyes,Brown Hair,Male Characters,NaN,Living Characters,1565.0,"1959, October",1959.0
3,1659,James Gordon (New Earth),\/wiki\/James_Gordon_(New_Earth),Public Identity,Good Characters,Brown Eyes,White Hair,Male Characters,NaN,Living Characters,1316.0,"1987, February",1987.0
4,1576,Richard Grayson (New Earth),\/wiki\/Richard_Grayson_(New_Earth),Secret Identity,Good Characters,Blue Eyes,Black Hair,Male Characters,NaN,Living Characters,1237.0,"1940, April",1940.0


In [14]:
dc = dc_data[['page_id','name','ID','ALIGN','SEX','ALIVE','APPEARANCES','YEAR']]

In [24]:
dc['alias'] = dc['name'].apply(get_alias)

In [27]:
dc.head()

,page_id,name,ID,ALIGN,SEX,ALIVE,APPEARANCES,YEAR,alias
0,1422,Batman (Bruce Wayne),Secret Identity,Good Characters,Male Characters,Living Characters,3093.0,1939.0,Batman
1,23387,Superman (Clark Kent),Secret Identity,Good Characters,Male Characters,Living Characters,2496.0,1986.0,Superman
2,1458,Green Lantern (Hal Jordan),Secret Identity,Good Characters,Male Characters,Living Characters,1565.0,1959.0,Lantern
3,1659,James Gordon (New Earth),Public Identity,Good Characters,Male Characters,Living Characters,1316.0,1987.0,Gordon
4,1576,Richard Grayson (New Earth),Secret Identity,Good Characters,Male Characters,Living Characters,1237.0,1940.0,Grayson


In [28]:
dc['true_name'] = dc['name'].apply(get_name)

In [29]:
dc.head()

,page_id,name,ID,ALIGN,SEX,ALIVE,APPEARANCES,YEAR,alias,true_name
0,1422,Batman (Bruce Wayne),Secret Identity,Good Characters,Male Characters,Living Characters,3093.0,1939.0,Batman,Bruce Wayne
1,23387,Superman (Clark Kent),Secret Identity,Good Characters,Male Characters,Living Characters,2496.0,1986.0,Superman,Clark Kent
2,1458,Green Lantern (Hal Jordan),Secret Identity,Good Characters,Male Characters,Living Characters,1565.0,1959.0,Lantern,Hal Jordan
3,1659,James Gordon (New Earth),Public Identity,Good Characters,Male Characters,Living Characters,1316.0,1987.0,Gordon,New Earth
4,1576,Richard Grayson (New Earth),Secret Identity,Good Characters,Male Characters,Living Characters,1237.0,1940.0,Grayson,New Earth


In [12]:
comics_df.to_csv('pd_comics.csv',index=False)

In [13]:
marv = marvel_data[['ALIGN','ALIVE','APPEARANCES','SEX','YEAR','name','page_id']]

In [14]:
marv.shape

(16376, 7)

In [15]:
engine = create_engine('sqlite:///comics.db')
#Base.metadata.create_all(engine)

In [16]:
#file_name = 'client_db.csv'
#df = pandas.read_csv(file_name)
marv.to_sql('marvel',con=engine, if_exists='replace')

In [9]:
#look at data for marvel

marvel_df.head()
#concat data

#filter data by marvel
#filter data by dc

#timeline of women in dc or marvel 
#how many female characters are good or evil
#count the number of female to male characters
#aggregating data pivoit tables

,page_id,name,urlslug,ID,ALIGN,EYE,HAIR,SEX,GSM,ALIVE,APPEARANCES,FIRST APPEARANCE,Year
0,1678,Spider-Man (Peter Parker),\/Spider-Man_(Peter_Parker),Secret Identity,Good Characters,Hazel Eyes,Brown Hair,Male Characters,NaN,Living Characters,4043.0,Aug-62,1962.0
1,7139,Captain America (Steven Rogers),\/Captain_America_(Steven_Rogers),Public Identity,Good Characters,Blue Eyes,White Hair,Male Characters,NaN,Living Characters,3360.0,Mar-41,1941.0
2,64786,"Wolverine (James \""Logan\"" Howlett)",\/Wolverine_(James_%22Logan%22_Howlett),Public Identity,Neutral Characters,Blue Eyes,Black Hair,Male Characters,NaN,Living Characters,3061.0,Oct-74,1974.0
3,1868,"Iron Man (Anthony \""Tony\"" Stark)",\/Iron_Man_(Anthony_%22Tony%22_Stark),Public Identity,Good Characters,Blue Eyes,Black Hair,Male Characters,NaN,Living Characters,2961.0,Mar-63,1963.0
4,2460,Thor (Thor Odinson),\/Thor_(Thor_Odinson),No Dual Identity,Good Characters,Blue Eyes,Blond Hair,Male Characters,NaN,Living Characters,2258.0,Nov-50,1950.0


In [7]:
marvel_df[~marvel_df['APPEARANCES'].isna()]

,page_id,name,urlslug,ID,ALIGN,EYE,HAIR,SEX,GSM,ALIVE,APPEARANCES,FIRST APPEARANCE,Year
0,1678,Spider-Man (Peter Parker),\/Spider-Man_(Peter_Parker),Secret Identity,Good Characters,Hazel Eyes,Brown Hair,Male Characters,NaN,Living Characters,4043.0,Aug-62,1962.0
1,7139,Captain America (Steven Rogers),\/Captain_America_(Steven_Rogers),Public Identity,Good Characters,Blue Eyes,White Hair,Male Characters,NaN,Living Characters,3360.0,Mar-41,1941.0
2,64786,"Wolverine (James \""Logan\"" Howlett)",\/Wolverine_(James_%22Logan%22_Howlett),Public Identity,Neutral Characters,Blue Eyes,Black Hair,Male Characters,NaN,Living Characters,3061.0,Oct-74,1974.0
3,1868,"Iron Man (Anthony \""Tony\"" Stark)",\/Iron_Man_(Anthony_%22Tony%22_Stark),Public Identity,Good Characters,Blue Eyes,Black Hair,Male Characters,NaN,Living Characters,2961.0,Mar-63,1963.0
4,2460,Thor (Thor Odinson),\/Thor_(Thor_Odinson),No Dual Identity,Good Characters,Blue Eyes,Blond Hair,Male Characters,NaN,Living Characters,2258.0,Nov-50,1950.0
5,2458,Benjamin Grimm (Earth-616),\/Benjamin_Grimm_(Earth-616),Public Identity,Good Characters,Blue Eyes,No Hair,Male Characters,NaN,Living Characters,2255.0,Nov-61,1961.0
6,2166,Reed Richards (Earth-616),\/Reed_Richards_(Earth-616),Public Identity,Good Characters,Brown Eyes,Brown Hair,Male Characters,NaN,Living Characters,2072.0,Nov-61,1961.0
7,1833,Hulk (Robert Bruce Banner),\/Hulk_(Robert_Bruce_Banner),Public Identity,Good Characters,Brown Eyes,Brown Hair,Male Characters,NaN,Living Characters,2017.0,May-62,1962.0
8,29481,Scott Summers (Earth-616),\/Scott_Summers_(Earth-616),Public Identity,Neutral Characters,Brown Eyes,Brown Hair,Male Characters,NaN,Living Characters,1955.0,Sep-63,1963.0
9,1837,Jonathan Storm (Earth-616),\/Jonathan_Storm_(Earth-616),Public Identity,Good Characters,Blue Eyes,Blond Hair,Male Characters,NaN,Living Characters,1934.0,Nov-61,1961.0


In [11]:
#learn how to subset nan/na and fill the with zero
#marvel_df['APPEARANCES'].isna() ==True

In [19]:
marvel_df.count()

page_id             16376
name                16376
urlslug             16376
ID                  12606
ALIGN               13564
EYE                  6609
HAIR                12112
SEX                 15522
GSM                    90
ALIVE               16373
APPEARANCES         15280
FIRST APPEARANCE    15561
Year                15561
dtype: int64

In [17]:
marvel_df[marvel_df['SEX'] == 'Male Characters']['SEX'].count()

11638

In [18]:
marvel_df[marvel_df['SEX'] == 'Female Characters']['SEX'].count()

3837

In [ ]:
df = df.max(axis=1)


In [25]:
marvel_df['APPEARANCES'].max(axis=1)

ValueError: No axis named 1 for object type <class 'pandas.core.series.Series'>

In [24]:
#Who has the most apperances in the marvel universe
marvel_df[marvel_df['APPEARANCES'].max()]

KeyError: 4043

In [ ]:
#make year a date